In [1]:
import pandas as pd
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from difflib import SequenceMatcher
import multiprocessing
import jieba


In [2]:
# Load the datasets
df_gemini = pd.read_csv("results_data/experimental_design_results_gemini-2.0-flash-thinking-exp_reprocessed.csv")
df = df_gemini

# Print the first few rows and info for each DataFrame
print("Gemini DataFrame:")
print(df.info())


Gemini DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   abstract    267 non-null    object
 1   Repetition  267 non-null    int64 
 2   ZH_EN       266 non-null    object
 3   EN_ZH       267 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.5+ KB
None


In [ ]:
# Tokenization function for Chinese texts using Jieba
def tokenize_chinese_jieba(text):
    # Use Jieba to cut the Chinese text into words
    return list(jieba.cut(text))

# Function to calculate BLEU (using unigram and bigram)
def calculate_bleu(candidate_tokens, reference_tokens):
    # We use weights (0.5, 0.5) for unigrams and bigrams
    try:
        return sentence_bleu([reference_tokens], candidate_tokens, weights=(0.5, 0.5, 0, 0))
    except Exception as e:
        return 0

# Function to calculate CHRF (character n-gram F-score)
def calculate_chrf(candidate_tokens, reference_tokens):
    candidate_str = "".join(candidate_tokens)
    reference_str = "".join(reference_tokens)
    # Calculates the ratio between the intersection and the union of the characters of the reference
    return len(set(candidate_str) & set(reference_str)) / len(set(reference_str)) if len(set(reference_str)) > 0 else 0

# Function to calculate TER using TF-IDF vectorization and mean squared error
def calculate_ter(candidate_text, reference_text):
    vectorizer = TfidfVectorizer() #token_pattern=r"(?u)\b\w+\b" - Removed token pattern because it is not needed for chinese
    try:
        tfidf_matrix = vectorizer.fit_transform([candidate_text, reference_text])
        return mean_squared_error(tfidf_matrix[0].toarray(), tfidf_matrix[1].toarray())
    except Exception as e:
        return 0

# Function to calculate Semantic Similarity (TF-IDF + cosine)
def calculate_semantic_similarity(original, translated):
    vectorizer = TfidfVectorizer() #token_pattern=r"(?u)\b\w+\b" - Removed token pattern because it is not needed for chinese
    try:
        tfidf_matrix = vectorizer.fit_transform([original, translated])
        return cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    except Exception as e:
        return 0

# Apply metrics to each row of the DataFrame
def calculate_metrics_for_row(row):
    original_text = row['abstract']
    back_translation = row['EN_ZH']

    # Tokenize the texts using Jieba
    original_tokens = tokenize_chinese_jieba(original_text)
    translated_tokens = tokenize_chinese_jieba(back_translation)

    # Calculate the metrics:
    bleu_value = calculate_bleu(translated_tokens, original_tokens)
    chrf_value = calculate_chrf(translated_tokens, original_tokens)
    ter_value = calculate_ter("".join(translated_tokens), "".join(original_tokens))
    semantic_similarity = calculate_semantic_similarity(original_text, back_translation)

    return pd.Series([bleu_value, chrf_value, ter_value, semantic_similarity])

def calculate_metrics_for_df(df):
    # tqdm.pandas(desc="Calculating metrics") # Removed progress bar
    return df.apply(calculate_metrics_for_row, axis=1)

def apply_parallel(df, func, n_cores=multiprocessing.cpu_count()):
    df_split = np.array_split(df, n_cores)
    pool = multiprocessing.Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

df[['BLEU', 'CHRF', 'TER', 'Semantic Similarity']] = apply_parallel(df, calculate_metrics_for_df)

# Display the results:
print(df[['abstract', 'EN_ZH', 'BLEU', 'CHRF', 'TER', 'Semantic Similarity']].head())

# Calculate and print the mean of the metrics
print("\nMean Metrics:")
print(df[['BLEU', 'CHRF', 'TER', 'Semantic Similarity']].mean())


/mnt/4d4f90e5-f220-481e-8701-f0a546491c35/arquivos/projetos/bt-conference/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Loading model fro

                                            abstract  \
0  文章首先阐述了工程教育专业认证与应用化学专业生产实习课程的关联，然后论述了工程教育专业认证背...   
1  文章首先阐述了工程教育专业认证与应用化学专业生产实习课程的关联，然后论述了工程教育专业认证背...   
2  文章首先阐述了工程教育专业认证与应用化学专业生产实习课程的关联，然后论述了工程教育专业认证背...   
3  “天然药物化学”是高等学校药学及相关专业的必修课程。课程章节内容多、理论性强,学生学习面临较...   
4  “天然药物化学”是高等学校药学及相关专业的必修课程。课程章节内容多、理论性强,学生学习面临较...   

                                               EN_ZH      BLEU      CHRF  \
0  文章首先阐述了工程教育专业认证与应用化学专业生产实习课程之间的联系。接着，探讨了在工程教育专...  0.693889  0.909091   
1  文章首先阐述了工程教育专业认证与应用化学专业生产实习课程之间的关系。其次，探讨了在工程教育专...  0.694830  0.909091   
2  文章首先阐述工程教育专业认证与应用化学生产实习课程的关系。然后探讨工程教育专业认证背景下应用...  0.698251  0.909091   
3  天然产物化学是高等院校药学及相关专业的必修课。课程内容章节丰富且理论性强，学生在学习中面临诸...  0.413904  0.890110   
4  天然药物化学是高校药学及相关专业的必修课。该课程内容广泛且理论性强，对学生的学习造成挑战。因...  0.427470  0.813187   

        TER  Semantic Similarity  
0  0.141732             0.078745  
1  0.121337             0.150640  
2  0.148285             0.184432  
3  0.072720             0.127360  
4  0.069251             0.16898

In [8]:
df.to_csv("results_metrics/results_metrics.csv", index=False)
